In [1]:
import os
import cv2
import dlib
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [2]:
# Load pre-trained models for landmark detection
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [3]:
def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])
        return landmarks
    return None

In [4]:
def calculate_angle(p1, p2, p3):
    v1 = p1 - p2
    v2 = p3 - p2
    return np.degrees(np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))))

In [5]:
def extract_features(landmarks):
    features = []
    
    # Basic distances
    eye_distance = np.linalg.norm(landmarks[36] - landmarks[45])
    nose_chin_distance = np.linalg.norm(landmarks[33] - landmarks[8])
    jaw_width = np.linalg.norm(landmarks[0] - landmarks[16])
    face_length = np.linalg.norm(landmarks[8] - landmarks[27])
    cheekbones = np.linalg.norm(landmarks[1] - landmarks[15])
    forehead = np.linalg.norm(landmarks[19] - landmarks[24])
    
    # Angles
    jaw_angle = calculate_angle(landmarks[0], landmarks[8], landmarks[16])
    cheekbone_angle = calculate_angle(landmarks[1], landmarks[27], landmarks[15])
    forehead_angle = calculate_angle(landmarks[19], landmarks[27], landmarks[24])
    
    # Ratios
    jaw_to_cheekbone_ratio = jaw_width / cheekbones
    face_length_to_width_ratio = face_length / cheekbones
    
    # New features
    chin_angle = calculate_angle(landmarks[6], landmarks[8], landmarks[10])
    temple_to_chin_ratio = np.linalg.norm(landmarks[0] - landmarks[16]) / face_length
    
    features = [eye_distance, nose_chin_distance, jaw_width, face_length, cheekbones, forehead,
                jaw_angle, cheekbone_angle, forehead_angle, jaw_to_cheekbone_ratio,
                face_length_to_width_ratio, chin_angle, temple_to_chin_ratio]
    
    return features

In [6]:
# Main directory path
main_dir = r'E:\archive (1)\FaceShape Dataset\training_set'

In [7]:
# Load dataset
feature_list, label_list = load_dataset(main_dir)

NameError: name 'load_dataset' is not defined

In [ ]:
# Create DataFrame
columns = ['eye_distance', 'nose_chin_distance', 'jaw_width', 'face_length', 'cheekbones', 'forehead',
           'jaw_angle', 'cheekbone_angle', 'forehead_angle', 'jaw_to_cheekbone_ratio',
           'face_length_to_width_ratio', 'chin_angle', 'temple_to_chin_ratio']

In [ ]:
df = pd.DataFrame(feature_list, columns=columns)

In [ ]:
df['face_shape'] = label_list

In [ ]:
# Save to CSV
df.to_csv('face_shape_features_knn.csv', index=False)

In [ ]:
# Load the dataset
data = pd.read_csv('face_shape_features_knn.csv')

In [ ]:
# Separate features and labels
X = data.drop('face_shape', axis=1)
y = data['face_shape']

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
#Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# KNN with GridSearchCV
knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

In [ ]:
knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn, param_grid=knn_params, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
# Best model
best_knn = grid_search.best_estimator_

In [ ]:
#Predict and evaluate
y_pred = best_knn.predict(X_test)

In [ ]:
#Print classification report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Save the model
joblib.dump(best_knn, 'knn_face_shape_model.pkl')
joblib.dump(label_encoder, 'label_encoder_knn.pkl')
joblib.dump(scaler, 'scaler_knn.pkl')